### Faire tous les imports nécessaires

In [186]:
import pandas as pd
import string
from nltk.tokenize import sent_tokenize
import re

### Importer les données

In [187]:
# les components
test_components_data = pd.read_csv('./data/test_components.csv')
train_components_data = pd.read_csv('./data/train_components.csv')
validation_components_data = pd.read_csv('./data/validation_components.csv')

# ajouts des 2 colonnes de contexte aux components
test_components_data['context1'] = ''
test_components_data['context2'] = ''
train_components_data['context1'] = ''
train_components_data['context2'] = ''
validation_components_data['context1'] = ''
validation_components_data['context2'] = ''

# les speechs
test_speeches_data = pd.read_csv('./data/test_speeches.csv')
train_speeches_data = pd.read_csv('./data/train_speeches.csv')
validation_speeches_data = pd.read_csv('./data/validation_speeches.csv')

# données de statistiques
size_components_data = len(test_components_data) + len(train_components_data) + len(validation_components_data)
print("Nombre de composants : ", size_components_data)

Nombre de composants :  32988


Petite fonction pour les resultats intermédiaires

In [188]:
# fonction permettant de retourner string permettant un print en couleur sur un pourcentage en python
# plus le pourcentage est proche de 0 plus il sera rouge, plus il est proche de 100 plus il sera vert
# le pourcentage sera calculé en fonction de la taille du dataset et d'un chiffre donné
def printResultColor(size_dataset, number):
    pourcentage = number / size_dataset * 100
    if pourcentage < 1:
        # print vert
        return("\033[92m {0:0.3f}%\033[0m".format(pourcentage))
    elif pourcentage < 5:
        # print orange
        return("\033[93m {0:0.3f}%\033[0m".format(pourcentage))
    else:
        # print rouge
        return("\033[91m {0:0.3f}%\033[0m".format(pourcentage))

### Implémentation de la premiere fonction

Cette fonction doit trouver le Text de tous components dans les Speeches. Une fois chaque Text trouvé, il faut completer le context 1 et 2.

Cette fonction renvoie une liste de tous les identifiants des components qui n'ont pas été trouvés et une liste de tous les composants qui ont été trouvés plusieur fois (multiple speeches).

In [189]:
def find1Component(dataset_component, dataset_speeches):
    not_found = []
    multiple_speech = []
    for index, row in dataset_component.iterrows():
        textToFind = row.Text
        speeches = dataset_speeches[dataset_speeches['Speech'].str.find(textToFind) != -1]
        # verifier que le component est bien dans un speech
        if len(speeches) == 0:
            not_found.append(index)
        elif len(speeches) == 1:
            # trouver le contexte 1 et 2
            # tokenizer le speech à l'aide de nltk
            sentences = sent_tokenize(speeches.iloc[0].Speech)
            # trouver la position de la phrase contenant le component
            position = -1
            for i in range(len(sentences)):
                if sentences[i].find(textToFind) != -1:
                    position = i
                    break
            if position == -1:
                # peut arriver si le component est en plusieurs phrases et donc la tokenizer ne le trouve pas
                not_found.append(index)
            else:
                # trouver le contexte 1
                if position > 0:
                    dataset_component.at[index, 'context1'] = sentences[position - 1]
                # ajouter la phrase contenant le component au contexte 1
                dataset_component.at[index, 'context1'] += sentences[position]
                # ajouter la phrase qui suit le component au contexte 1
                if position < len(sentences) - 1:
                    dataset_component.at[index, 'context1'] += sentences[position + 1]
                
                # mettre tout le speech dans le contexte 2
                dataset_component.at[index, 'context2'] = speeches.iloc[0].Speech
        else:
            # plusieurs speeches contiennent le component
            # il va falloir verifier l'id du speech et de la section pour verifier si c'est le bon
            multiple_speech.append(index)
    return not_found, multiple_speech

In [190]:
# tester la fonction
not_founded_test = []
multiple_founded_test = []
not_founded_test, multiple_founded_test = find1Component(test_components_data, test_speeches_data)
print("Nombre de composants test non trouvés: {} -> {}".format(len(not_founded), printResultColor(len(test_components_data), len(not_founded))))
print("Nombre de composants test trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded), printResultColor(len(test_components_data), len(multiple_founded))))

not_founded_train = []
multiple_founded_train = []
not_founded_train, multiple_founded_train = find1Component(train_components_data, train_speeches_data)
print("Nombre de composants train non trouvés: {} -> {}".format(len(not_founded_train), printResultColor(len(train_components_data), len(not_founded_train))))
print("Nombre de composants train trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_train), printResultColor(len(train_components_data), len(multiple_founded_train))))

not_founded_validation = []
multiple_founded_validation = []
not_founded_validation, multiple_founded_validation = find1Component(validation_components_data, validation_speeches_data)
print("Nombre de composants validation non trouvés: {} -> {}".format(len(not_founded_validation), printResultColor(len(validation_components_data), len(not_founded_validation))))
print("Nombre de composants validation trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_validation), printResultColor(len(validation_components_data), len(multiple_founded_validation))))


Nombre de composants test non trouvés: 102 ->  1.074%
Nombre de composants test trouvés dans plusieurs discours: 1030 ->  10.850%

Nombre de composants train non trouvés: 306 ->  1.793%
Nombre de composants train trouvés dans plusieurs discours: 777 ->  4.552%

Nombre de composants validation non trouvés: 109 ->  1.696%
Nombre de composants validation trouvés dans plusieurs discours: 144 ->  2.241%



### Implémentation de la deuxième fonction

Cette fonction doit trouver le Text de tous les components qui sont en plusieurs phrases et qui n'ont pas été trouvés précédements, avec ou sans la ponctuation. Le resultat renvoyé sera similaire à celui de la fonction précédente.

In [203]:
def find2Components(idsComponents, dataset_component, dataset_speeches):
    not_found = []
    # pour chaque id de component
    for idComponent in idsComponents:
        # get le text du component
        textToFind = dataset_component.iloc[idComponent].Text
        # enlever les ponctuations
        textToFind = re.sub(r'[\.!\?]','',textToFind)
        # find the text in the speeches, but this time we remove the ponctuations of the speeches using replace function of pandas
        speeches = dataset_speeches[dataset_speeches['Speech'].str.replace(r'[\.!\?]', '', regex=True).str.find(textToFind) != -1]
        if(len(speeches) == 1):
            # si on a trouvé le component dans un seul speech
            # TODO pour cet après midi
            pass
        else:
            not_found.append(idComponent)

In [204]:
find2Components(not_founded_train, train_components_data, train_speeches_data)